In [ ]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [ ]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [ ]:
%matplotlib notebook

In [ ]:
df = pd.read_excel("赛题1测试数据集/购地-市场交易-土地转让.xlsx", index_col="企业编号").drop_duplicates()
df

In [ ]:
def to_land_level(cell):
    if not isinstance(cell, str):
        return cell
    if '一级' in cell and '十一级' not in cell:
        return '一级'
    if '8' in cell:
        return '八级'
    if '1' in cell:
        return '一级'
    if '4' in cell:
        return '四级'
    if '2' in cell:
        return '二级'
    if '5' in cell:
        return '五级'
    return cell
    
    
land_level = pd.get_dummies(df['土地级别'].map(to_land_level), prefix='土地级别').groupby('企业编号').sum().T
land_level = land_level.reindex(columns=list(range(4001,4526))).fillna(0).reindex(['土地级别_一级', '土地级别_七级', '土地级别_三级', '土地级别_九级', '土地级别_二级', '土地级别_五级',
       '土地级别_八级', '土地级别_六级', '土地级别_十一级', '土地级别_十三等级', '土地级别_十二级', '土地级别_十五等级',
       '土地级别_十八等级', '土地级别_十四等级', '土地级别_十级', '土地级别_四级', '土地级别_未评估地区']).fillna(0)
land_level

In [ ]:
lst = df['土地使用权类型'].value_counts()[df['土地使用权类型'].value_counts() == 1].index


def to_land_use_right_type(cell):
    if not isinstance(cell, str):
        return cell
    if cell not in lst:
        return cell


land_use_right_type = pd.get_dummies(df['土地使用权类型'].map(to_land_use_right_type), prefix='土地使用权类型').groupby('企业编号').sum().T
land_use_right_type = land_use_right_type.reindex(columns=list(range(4001,4526))).fillna(0).reindex(['土地使用权类型_出让', '土地使用权类型_出让(协议)', '土地使用权类型_出让国有土地', '土地使用权类型_出让国有土地使用权',
       '土地使用权类型_出让国有建设用地使用权', '土地使用权类型_出让地', '土地使用权类型_划拨', '土地使用权类型_协议',
       '土地使用权类型_协议出让', '土地使用权类型_国有', '土地使用权类型_国有、出让', '土地使用权类型_国有出让',
       '土地使用权类型_国有出让土地', '土地使用权类型_国有出让地', '土地使用权类型_国有出让用地', '土地使用权类型_国有土地使用权',
       '土地使用权类型_国有土地使用权转让', '土地使用权类型_属出让', '土地使用权类型_挂牌出让', '土地使用权类型_授权经营',
       '土地使用权类型_租赁', '土地使用权类型_转让', '土地使用权类型_集体土地使用权']).fillna(0)
land_use_right_type

In [ ]:
transfer_method = pd.get_dummies(df['转让方式'], prefix='转让方式').groupby('企业编号').sum().T
transfer_method = transfer_method.reindex(columns=list(range(4001,4526))).fillna(0).reindex(['转让方式_1', '转让方式_3', '转让方式_6', '转让方式_7', '转让方式_买卖', '转让方式_交换',
       '转让方式_作价出资入股', '转让方式_其他', '转让方式_继承']).fillna(0)
transfer_method

In [ ]:
pd.concat([land_level, land_use_right_type, transfer_method]).to_pickle('购地-市场交易-土地转让-testing.pickle')